Following watermarking method from https://arxiv.org/abs/1701.04082 
Consists in encode message in layers trainable weights
Input is used as key for decyphering.

Define model structure and input given fixed seed (this code is given for the challenge).

In [1]:
import torch
import torch.nn as nn
torch.manual_seed(1234)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Prendre une clef aléatoire -> bien penser à la sauvegarder !
X = torch.randn(144, 150)

Load pretrained model with weights (given by challenge) and format input

In [2]:
model = Net()
# X = X.unsqueeze(0)

model.load_state_dict(torch.load('404_model.pth'))

<All keys matched successfully>

Extract and flatten weights from second convolution layer (following paper).
Second layer is chosen to match size of input $X$ when flattened.

In [3]:
w = model.conv2.weight.detach().numpy().mean(0)
w = w.flatten()

1. Calculate sum of linear combinations between input and weights $B = \sum_{i=1}^n x_i \times w$
2. Apply step function:
  - 1 if $B > 0$
  - 0 otherwise

In [4]:
import numpy as np

bits = []
for i in range(X.shape[0]):
  b = np.sum(X[i].numpy() * w)
  bits.append(str(int(b >= 0)))
''.join(bits)

'001101000011000000110100010000110101010001000110011110110101000000110100011100110101111101110100001100000111010101000011011010000011001101111101'

From resulting sequence, convert binary string to ASCII (https://www.rapidtables.com/convert/number/binary-to-ascii.html) to get:

404CTF{P4s_t0uCh3}